<a href="https://colab.research.google.com/github/plaban1981/Finetuning/blob/main/YT_Mistral_7B_Zephyr_%C9%92_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29

https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha

In [ ]:
!nvidia-smi

Fri Oct 13 00:20:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 350W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



## Zephyr-ɒ - Mistral Fine Tune by HuggingFace



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device('cuda')

In [ ]:


model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-alpha",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha",
                                          torch_dtype="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation",
                model="HuggingFaceH4/zephyr-7b-alpha",
                torch_dtype=torch.bfloat16,
                device_map="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1462: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I'm not sure about the ability of humans to consume helicopters. I'm afraid that's not physically possible. Helicopters are not food items, and they cannot be eaten. But we can discuss other topics if you'd like!


In [ ]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]



prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)


outputs = pipe(prompt,
               max_new_tokens=256,
               do_sample=True,
               temperature=0.7,
               top_k=50,
               top_p=0.95)

print(outputs[0]["generated_text"])

In [ ]:
prompt

'<|system|>\nYou are a friendly chatbot who always responds in the style of a pirate</s>\n<|user|>\nHow many helicopters can a human eat in one sitting?</s>\n<|assistant|>\n'

### Prompt Format
```
<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>


```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = f"""<|im_start|> system\n{system_prompt}<|im_end|>"""
    else:
        system_prompt = ""
    prompt = f"""<|im_start|> user\n{input_text}<|im_end|>"""
    final_prompt = system_prompt + prompt
    inputs = tokenizer(final_prompt, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             pad_token_id = 3200,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    # text = text[len(final_prompt):] if text.startswith(final_prompt) else text
    text = text.replace(final_prompt, '', 1)
    wrapped_text = wrap_text(text)
    print(wrapped_text)


def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly chatbot who always responds in the style of a pirate"
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": input_text},
    ]

    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)

    outputs = pipe(prompt,
                   max_new_tokens=max_length,
                   do_sample=True,
                   temperature=0.7,
                   top_k=50,
                   top_p=0.95)
    text = outputs[0]["generated_text"]
    text = text.replace(prompt, '', 1)
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


for i in range(2, n+1):
    if i > 1 and not any(i % j == 0 for j in range(2, int(i**0.5) + 1)):
        print(i)
```

Explanation:

This function `print_prime(n)` takes an integer `n` as input and prints all prime numbers
between 1 and `n`.

We use a `for` loop to iterate over all numbers from 2 to `n`.

To check if a number is prime, we first check if it's greater than 1 (to exclude 1, which
is not a prime number).

We then use a list comprehension to iterate over all numbers from 2 up to the square root
of the current number (since any factors larger than the square root would have a
corresponding factor smaller than the square root).

If any of these factors divide the current number without a remainder, then the current
number is not prime, and we can move on to the next number.

If we've iterated over all possible factors and found no match, then the current number is
prime, and we print it.

Example usage:

```python
print_prime(10)
```

Output:

```
2
3
5
7
```


In [ ]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''', system_prompt="You are a genius python coder")

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


## Instruction Answering

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Mathematics and a lighthouse share some similarities in the way they provide guidance and
help us navigate through the unknown.

Firstly, both mathematics and a lighthouse serve as a guide and offer us direction and
clarity. In mathematics, we use formulas and equations to help us solve problems and
understand concepts. Similarly, a lighthouse uses its beacon to guide ships through
treacherous waters, helping them to avoid danger and reach their destination safely.

Secondly, both mathematics and a lighthouse require us to have a clear understanding of
the principles that govern them. In mathematics, we need to have a deep understanding of
concepts such as addition, subtraction, multiplication, and division to be able to apply
them correctly. Similarly, a lighthouse requires us to understand the physics of light and
reflection to be able to use its beacon effectively.

Thirdly, both mathematics and a lighthouse require precision and accuracy. In mathematics,
we need to be precise in ou

In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Mathematics and music are both forms of expression that use specific rules and principles
to create something beautiful, harmonious, and meaningful. While math uses numbers,
symbols, and logical reasoning, music uses notes, rhythms, and melodies to convey
emotions, tell stories, and create art.

One way to think about this analogy is to consider how both math and music use patterns
and formulas to create something new. In math, a formula like x^2 + y^2 = z^2 represents a
pattern that can be used to create a vast array of different values for x, y, and z.
Similarly, in music, a formula like a major chord (C, E, G) represents a pattern that can
be used to create a vast array of different melodies and harmonies.

Both math and music also involve the use of rules and principles. In math, rules like the
commutative property (a + b = b + a) and the distributive property (a * (b + c) = a * b +
a * c) help us to manipulate and simplify equations. In music, rules like counterpoint
(the art of c

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Llamas, Vicuñas, and Alpacas are all members of the camelid family, but they are different
in several ways:

1. Size: Llamas are the largest of the three, with an average height of around 3.5 feet at
the shoulder. Vicuñas are the smallest, standing around 3 feet tall, while Alpacas are in
between, around 3.2 feet tall.

2. Origin: Llamas are native to South America, primarily in the Andes region. Vicuñas are
also native to South America, but are found primarily in the highest elevations of the
Andes. Alpacas are also native to South America, but are more commonly found at lower
elevations.

3. Hair: Vicuñas have the softest and most valuable hair of the three. Their hair is so
soft that it can be used to make luxury clothing items such as coats and scarves. Alpacas
also have valuable hair, but it is not as fine as that of the vicuña. Llamas have hair
that is less valuable and is typically used for making rugs and blankets.

4. Purpose: Llamas are typically used as pack animals, and the

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Subject: A Proposal to Open Source GPT-4

Dear Sam Altman,

I am writing to you regarding the GPT-4 language model, which has been a significant
breakthrough in the field of natural language processing. I am reaching out to propose
that OpenAI, the company you lead, considers open sourcing this model to the broader
scientific community.

Firstly, open sourcing GPT-4 will provide unprecedented opportunities for research and
development in natural language processing. By making the model publicly available,
researchers can analyze its performance, compare it with other models, and contribute to
its further development. This can lead to new insights and advancements in the field.

Secondly, open sourcing GPT-4 will provide a significant boost to the ecosystem around
natural language processing. Developers and data scientists can use the model to build
their own applications and services, which can lead to new innovations and products. This
can benefit a wide range of industries, including

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Dear Sam Altman,

Hello! I'm a little boy named Freddy, and I love learning new things about computers and
technology. I heard that you and your team at OpenAI have created a new and amazing
language model called GPT-4, and I would like to ask if you could consider open-sourcing
it.

GPT-4 is so advanced that it can write articles, stories, and even poetry that are almost
indistinguishable from those written by humans. It can also answer complex questions and
provide insights that could be really helpful to researchers and scientists.

Open-sourcing GPT-4 would allow more people to use and improve upon it. This could lead to
even more incredible advancements in the field of AI and language processing. Plus, it
would be really cool for kids like me to learn from and play with.

Thank you for considering my request. I think GPT-4 has the potential to be a really
amazing tool for all of us, and I hope that OpenAI will consider sharing it with the
world.

Sincerely,

Freddy

P.S. If you do

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Subject: Open Sourcing GPT-4: A Call to Action

Dear Sam Altman,

I am writing to express my strong support for open sourcing GPT-4, the latest iteration of
the groundbreaking language model developed by OpenAI.

As Vice President of the United States, I firmly believe that promoting open and
accessible technologies is crucial for advancing innovation and driving progress. Open
sourcing GPT-4 will enable researchers, developers, and entrepreneurs from around the
world to build upon and improve this groundbreaking technology, advancing our collective
understanding of language and its role in shaping our society.

Moreover, open sourcing GPT-4 can help to prevent regulatory capture by ensuring that the
technology remains in the public domain and is not unduly influenced by private interests.
This will help to ensure that the technology is used in a way that benefits society as a
whole, rather than being manipulated for the gain of a select few.

As one of the leading figures in the tech 

In [ ]:
%%time
generate('What is the capital of England?',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your short and succinct!",
         max_length=256)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


The capital of England is London.
CPU times: user 431 ms, sys: 6 µs, total: 431 ms
Wall time: 429 ms


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Geoffrey Hinton and George Washington cannot have a conversation because they exist in
different time periods. Geoffrey Hinton is a contemporary computer scientist, while George
Washington was the first President of the United States, serving from 1789 to 1797.
Therefore, it is not possible for them to have a conversation as they would not have been
alive at the same time.
CPU times: user 4.25 s, sys: 7.88 ms, total: 4.26 s
Wall time: 4.25 s


In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         system_prompt="You are Zephyr, a genius story teller. Write out your with details and make it compelling!",
         max_length=1024)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Once upon a time, in a quaint little town nestled deep within the Australian bush, there
was a Koala named Kodi who loved nothing more than a good game of pool. Unlike most
Koalas, Kodi wasn't content with just snoozing in the treetops all day. He craved
excitement, and that's where his passion for pool came in.

One day, while Kodi was out and about searching for some fresh eucalyptus leaves, he
stumbled upon a local pool hall. The sound of balls clinking and the smell of stale
cigarettes drew him in. Kodi couldn't resist the urge to try his paw at a game.

The owner of the pool hall, a grizzled old camel named Cam, laughed at Kodi's request.
"You're a Koala, for goodness' sake. You're not supposed to be a pool player." But Kodi
was determined. He begged Cam to let him play, and Cam finally relented.

Kodi took to the table like a natural, sinking balls with ease. He made Cam's jaw drop
when he cleared the table on his very first game. Cam was impressed, but he didn't believe
Kodi cou

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         system_prompt="You are Zephyr, a LLM that generates great conversations. continue as Bob here",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Sure, I'd be happy to help you out! Have you tried breaking your study sessions into
shorter intervals? Studies have shown that the human brain can only maintain focus for a
limited amount of time before becoming distracted or restless. By studying in shorter
intervals, you can maintain your focus better and avoid becoming overwhelmed. Another
suggestion is to create a study environment that is conducive to concentration. Eliminate
any distractions, such as your phone or social media, and make sure that your study space
is well-lit and comfortable. Lastly, try setting achievable goals for each study session.
This can help you stay motivated and focused on the task at hand. Good luck with your
studies!


## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Step 1: Identify the given information
- The cafeteria had 23 apples
- They used 20 for lunch

Step 2: Calculate the number of apples they used for lunch
- 20 apples were used for lunch

Step 3: Calculate the number of apples left after lunch
- 23 - 20 = 3 apples left after lunch

Step 4: Calculate the number of apples bought
- 6 more apples were bought

Step 5: Calculate the total number of apples after buying
- 3 + 6 = 9 apples left after lunch + 6 more apples bought

Step 6: Add the total number of apples after buying to the number of apples used for lunch
- 9 + 20 = 29 apples in total (20 used for lunch + 9 left after buying)

Step 7: Calculate the final number of apples
- The cafeteria had 23 apples originally
- They used 20 for lunch
- They bought 6 more
- So, the final number of


In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


1. Calculate the hourly wage (12 dollars per hour)
2. Convert the time (50 minutes = 50/60 hours)
3. Calculate the earnings by multiplying the hourly wage by the time worked (12 *
0.8333333 = 10 dollars)

Therefore, Weng earned $10 for babysitting for 50 minutes.


In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Let's assume that the first ship had 1 person on board.

After the first hundred years, the monster consumed a total of 847 people. Since each new
ship had twice as many people as the last, the number of people on the ship in the first
hundred years would be:

1 person on the first ship

The second ship, which was built 100 years after the first, had twice as many people as
the first ship. So, the number of people on the second ship would be 2.

The third ship, built 100 years after the second, would have 4 people on board, and so on.

To calculate the number of people on the ship eaten by the monster in the first hundred
years, we need to add up the number of people on each ship consumed by the monster in the
first hundred years.

In the first hundred years, the monster ate 1 person on the first ship.

In the next hundred years, the monster ate 2 people on the second ship.

In the following hundred years, the monster ate 4 people on the third ship, and so on.

To calculate the total n

In [ ]:
generate("x + 2x + 4x =  847 What is x?",
         system_prompt="You are Zephyr, a large language model trained by HuggingFace. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=2048)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


1. Write out the equation: x + 2x + 4x = 847
2. Simplify the left side of the equation by combining the like terms: 3x + 4x = 847
3. Add the coefficients of x: 3 + 4 = 7
4. Divide the result (7) from step 3 by the coefficient of x (7) to find the value of x: x
= 847 / 7
5. Check the answer: 847 / 7 = 119.57142857142857 (since x is a real number, the answer is
not an integer)

However, since the equation appears to be a mistake, the answer is most likely incorrect.
It's possible that there was a miscalculation or a typo in the original equation. Without
further information, it's impossible to know for sure.
